In [1]:
!pip install tensorflow
!pip install numpy

Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo2020/avx512, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/avx512, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo2020/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo2020/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo2020/avx512, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/avx512, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo2020/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo2020/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/

In [2]:
!pip install matplotlib

Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo2020/avx512, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/avx512, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo2020/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo2020/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
Processing /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/avx2/matplotlib-3.7.2+computecanada-cp39-cp39-linux_x86_64.whl
Processing /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo2020/avx2/Pillow-10.1.0+computecanada-cp39-cp39-linux_x86_64.whl
Processing /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic/pyparsing-3.0.9+computecanada-py3-none-any.whl
Processing /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/avx2/kiwisolver-1.4.5+computecana

In [7]:
import numpy as np
import sys
import datetime
import matplotlib.pyplot as plt

In [8]:
from tensorflow import keras
from tensorflow.keras.models import Model
import tensorflow as tf

In [5]:
import pathlib
import os

data_dir_train = pathlib.Path('./OCT2017 /train/')
data_dir_test = pathlib.Path('./OCT2017 /test/')
print(data_dir_train)
print(data_dir_test)
print(os.path.abspath(data_dir_train))

OCT2017 /train
OCT2017 /test
/project/60006/thomaslaboucheix/OCT2017 /train


In [6]:
batch_size = 3
img_height = 200
img_width = 200

train_data = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_train,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size,
)

val_data = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_test,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

class_names = train_data.class_names
print(class_names)

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_data.take(1):
  for i in range(3):
    ax = plt.subplot(1, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

num_classes = 4

model = tf.keras.Sequential([
    layers.experimental.preprocessing.Rescaling(1./255),
    layers.Conv2D(128, 4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(16, 4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

In [ ]:
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_auto_jit=2'

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

logdir="logs"

tensorboard_callback = keras.callbacks.TensorBoard(
    log_dir=logdir,
    histogram_freq=1,
    write_images=logdir,
    embeddings_data=train_data
)

In [ ]:
!pip install --upgrade tensorflow

In [ ]:
model.fit( 
    train_data,
    validation_data=val_data,
    epochs=15,
    callbacks=[tensorboard_callback]
)

In [ ]:
model.save('my_model.keras')

In [ ]:
#model.summary()

In [ ]:
test_loss, test_accuracy = model.evaluate(val_data)
print(f"Test Accuracy: {test_accuracy*100:.2f}%")

In [ ]:
#weights = model.get_weights()

# Afficher les poids
#for i in range(len(weights)):
#    print(f'Layer {i + 1} - Shape: {weights[i].shape}')
#    print(weights[i])

In [ ]:
# test l'ia avec

#from google.colab import files
from tensorflow.keras.preprocessing import image

img_path_cnv = './OCT2017 /test/CNV/CNV-1016042-1.jpeg'
img_path_dme = './OCT2017 /test/DME/DME-1081406-1.jpeg'
img_path_drusen = './OCT2017 /test/DRUSEN/DRUSEN-1083159-1.jpeg'
img_path_normal = './OCT2017 /test/NORMAL/NORMAL-1017237-1.jpeg'

def predict_picture(path):
    img = image.load_img(path, target_size=(200, 200))

    plt.imshow(img)
    plt.axis('off')
    plt.show()

    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    res = model.predict(img_array)
    print("res", res)
    predicted_class = np.argmax(res)
    print('predicted_class', predicted_class)
    class_probabilities = res[0]
    for i, prob in enumerate(class_probabilities):
        print(f"Probabilité de la classe {i}: {prob*100:.2f}%")
    # ['CNV', 'DME', 'DRUSEN', 'NORMAL']

print("cnv")
predict_picture(img_path_cnv)
print("dme")
predict_picture(img_path_dme)
print("drusen")
predict_picture(img_path_drusen)
print("normal")
predict_picture(img_path_normal)